## Parte 1

la entrada tiene la forma X N donde X es la dirección D (down), L (left), R (right), U (up) y N el número de pasos que hay que dar. esto genera un contorno, hay que averiguar el área total del contorno mas todo lo que queda en su interior.

In [1]:
def get_input(filename):
    instructions = []
    with open(filename) as file:
        for line in file:
            parts = line.strip().split(' ')
            direction = parts[0]
            steps = int(parts[1])
            color = parts[2].replace('(', '').replace(')', '')
            instructions.append((direction, steps, color))
    return instructions

In [3]:
def map_terrain(instructions):
    position = (0, 0)
    terrain = {position: '#'}
    
    for direction, steps, _ in instructions:
        
        if direction == 'R':
            for j in range(steps):
                position = (position[0], position[1] + 1)
                terrain[position] = '#'
        elif direction == 'L':
            for j in range(steps):
                position = (position[0], position[1] - 1)
                terrain[position] = '#'
        elif direction == 'D':
            for j in range(steps):
                position = (position[0] + 1, position[1])
                terrain[position] = '#'
        elif direction == 'U':
            for j in range(steps):
                position = (position[0] - 1, position[1])
                terrain[position] = '#'
    return terrain

In [5]:
def get_area(terrain):
    minx = min([x for x, _ in terrain.keys()])
    miny = min([y for _, y in terrain.keys()])
    maxx = max([x for x, _ in terrain.keys()])
    maxy = max([y for _, y in terrain.keys()])
    
    huecos = set(terrain.keys())
    
    for y in range(miny, maxy):
        dentro = False
        for x in range(minx, maxx):
            if (x, y) in huecos:
                if (x, y - 1) in huecos and (x, y + 1) in huecos:
                    dentro = not dentro
                elif (x, y + 1) in huecos and (x + 1, y) in huecos:
                    dentro = not dentro
                elif (x, y + 1) in huecos and (x - 1, y) in huecos:
                    dentro = not dentro
            else:
                if dentro:
                    huecos.add((x, y))
    return len(huecos)

In [6]:
instructions = get_input('test.txt')
terrain = map_terrain(instructions)
area = get_area(terrain)
area

62

In [8]:
instructions = get_input('input.txt')
terrain = map_terrain(instructions)
area = get_area(terrain)
area

53844

## Parte 2

había un error en las instrucciones, hay que trabajar solo con la parte hexadecimal, así, el último dígito hexadecimal codifica la dirección 0 = R, 1 = D, 2 = L y 3 = U. Los otros 5 dígitos hexadecimales codifican la distancia.

In [11]:
from shapely.geometry import Polygon

In [29]:
def get_area_hex(instructions):
    #0: R, 1: D, 2: L, 3: U
    dirs = {'0': (1, 0), '1': (0, 1), '2': (-1, 0), '3': (0, -1)}
    vs = []
    
    for _, _, color in instructions:
        d = color[-1]
        dist = int(color[1:-1], 16)
        vector = (dirs[d][0] * dist, dirs[d][1] * dist)
        vs.append(vector)
        
    #hace falta acumular porque los vectores parten todos desde el origen
    cumsum = (0, 0)
    puntos = []
    perimetro = 0
    
    for v in vs:
        cumsum = [cumsum[0] + v[0], cumsum[1] + v[1]]
        perimetro += abs(v[0]) + abs(v[1])
        puntos.append(cumsum)
        
    poli = Polygon(puntos)
    
    return int(poli.area + (perimetro / 2) + 1)

In [30]:
instructions = get_input('test.txt')
area = get_area_hex(instructions)
area

952408144115

In [31]:
instructions = get_input('input.txt')
area = get_area_hex(instructions)
area

42708339569950